# Yolo V9


### Get started
- Download dataset through RoboFlow API


In [2]:
import os
from roboflow import Roboflow
import cv2


# Initialize Roboflow and download the dataset
rf = Roboflow(api_key="Qg1dq8rp1RHvCPvgWSNZ")
project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(8)
dataset = version.download("yolov8", "../datasets/football-players-detection")

# Print the path where the dataset is saved
print("Dataset downloaded and saved at:", dataset)


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.2, to fix: `pip install ultralytics==8.0.196`
Dataset downloaded and saved at: <roboflow.core.dataset.Dataset object at 0x780cd5600220>


### Custom callback function for freezing of layers

In [3]:
def freeze_layer(trainer):
    model = trainer.model
    num_freeze = 10
    print(f"Freezing {num_freeze} layers")
    freeze = [f'model.{x}.' for x in range(num_freeze)]  # layers to freeze 
    for k, v in model.named_parameters(): 
        v.requires_grad = True  # train all layers 
        if any(x in k for x in freeze): 
            print(f'freezing {k}') 
            v.requires_grad = False 
    print(f"{num_freeze} layers are freezed.")

### Train model

In [4]:
from ultralytics import YOLO

# Build a YOLOv9c model from pretrained weight
model = YOLO('../pretrained-weights/yolov9c.pt')

# Display model information (optional)
model.info()

# Freeze the first 10 layers (backbone) of the model
model.add_callback("on_train_start", freeze_layer)

# Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data='../datasets/football-players-detection/data.yaml', epochs=1, imgsz=640)



YOLOv9c summary: 618 layers, 25590912 parameters, 0 gradients, 104.0 GFLOPs


(618, 25590912, 0, 104.02268160000003)

### Run inference

In [8]:
# Run inference with the YOLOv9c model on the 'bus.jpg' image

dir = '../datasets/football-players-detection/test/images/'
images = [dir + f for f in os.listdir(dir) if f.endswith('.jpg')]

results = model(images)


0: 384x640 23 persons, 877.1ms
1: 384x640 24 persons, 1 tv, 877.1ms
2: 384x640 19 persons, 1 tv, 877.1ms
3: 384x640 24 persons, 1 tv, 877.1ms
4: 384x640 19 persons, 877.1ms
5: 384x640 31 persons, 877.1ms
6: 384x640 23 persons, 1 tv, 877.1ms
7: 384x640 21 persons, 1 tv, 877.1ms
8: 384x640 23 persons, 877.1ms
9: 384x640 26 persons, 1 tv, 877.1ms
10: 384x640 24 persons, 1 tv, 877.1ms
11: 384x640 27 persons, 877.1ms
12: 384x640 19 persons, 1 tv, 877.1ms
Speed: 2.9ms preprocess, 877.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)


### Show inferenced image

In [7]:
# Visualize the results on the frame
annotated_frame = results[0].plot()
